In [ ]:
from detectron2.data import DatasetCatalog

# train_dataset = DatasetCatalog.get("your_dataset_name")  # registered dataset
# num_samples = len(train_dataset)

# batch_size = cfg.SOLVER.IMS_PER_BATCH
# num_epochs = 10  # or any dynamic value you want

# iters_per_epoch = num_samples // batch_size
# cfg.SOLVER.MAX_ITER = iters_per_epoch * num_epochs

# cfg.SOLVER.STEPS = (
#     int(cfg.SOLVER.MAX_ITER * 0.6),
#     int(cfg.SOLVER.MAX_ITER * 0.8),
# )


In [1]:
# from detectron2.config import get_cfg
# from detectron2.data import DatasetCatalog

# cfg = get_cfg()
# cfg.merge_from_file("your_config.yaml")

# # Register dataset beforehand
# num_samples = len(DatasetCatalog.get("your_dataset_name"))

# batch_size = cfg.SOLVER.IMS_PER_BATCH
# num_epochs = 10

# cfg.SOLVER.MAX_ITER = (num_samples // batch_size) * num_epochs
# cfg.SOLVER.STEPS = (
#     int(cfg.SOLVER.MAX_ITER * 0.6),
#     int(cfg.SOLVER.MAX_ITER * 0.8),
# )


In [1]:
import os

import _init_paths

import models.hrnet

from detectron2.modeling.meta_arch import META_ARCH_REGISTRY
from detectron2.modeling import build_backbone, BACKBONE_REGISTRY, Backbone

# from models.hrnet import build_hrnet_backbone

import torch
import torch.nn as nn
import torch.nn.functional as nnf

import os

BN_MOMENTUM = 0.1

@META_ARCH_REGISTRY.register()
class HRNet_SEG(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.num_masks = cfg.MODEL.NUM_MASKS
        self.align_corners = cfg.MODEL.ALIGN_CORNERS

        self.backbone = build_hrnet_backbone(cfg)

        fm_c = sum(cfg.MODEL.HRNET.STAGE4.NUM_CHANNELS)

        self.seg_head = nn.Sequential(
            nn.Conv2d(fm_c, 64, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(64, momentum=BN_MOMENTUM),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, self.num_masks, 1, stride=1, padding=0),
        )

        self.init_weights(cfg.MODEL.MODEL_WEIGHTS)
    
    def forward(self, x):
        x = self.backbone(x)

        x0_h, x0_w = x[0].size(2), x[0].size(3)
        x1 = nnf.interpolate(x[1], size=(x0_h, x0_w), mode='bilinear', align_corners=self.align_corners)
        x2 = nnf.interpolate(x[2], size=(x0_h, x0_w), mode='bilinear', align_corners=self.align_corners)
        x3 = nnf.interpolate(x[3], size=(x0_h, x0_w), mode='bilinear', align_corners=self.align_corners)

        x = torch.cat([x[0], x1, x2, x3], 1)
        x = self.seg_head(x)

        return x

    def init_weights(self, pretrained=""):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                nn.init.normal_(m.weight, std=0.001)
                for name, _ in m.named_parameters():
                    if name in ['bias']:
                        nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
        if os.path.isfile(pretrained):
            pretrained_state_dict = torch.load(pretrained)
            print('=> loading pretrained model {}'.format(pretrained))

            need_init_state_dict = {}
            for name, m in pretrained_state_dict.items():
                if name.split('.')[0] in self.pretrained_layers \
                   or self.pretrained_layers[0] == '*':
                    need_init_state_dict[name] = m
            self.load_state_dict(need_init_state_dict, strict=False)
        elif pretrained:
            print('=> please download pre-trained models first!')
            raise ValueError('{} is not exist!'.format(pretrained))

@BACKBONE_REGISTRY.register()
def build_hrnet_backbone(cfg, input_c=3):
    backbone = models.hrnet.HighResolutionNet(cfg, input_c)

    return backbone

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
from config import cfg
from detectron2.modeling import build_model


cfg_path = "/workspace/project/configs/hrnet/w32_only_complt.yaml"

cfg.merge_from_file(cfg_path)

cfg.MODEL.WEIGHTS = "/workspace/models/pytorch/imagenet/hrnet_w32-36af842e.pth"

model = build_model(cfg)

model

Config '/workspace/project/configs/hrnet/w32_only_complt.yaml' has no VERSION. Assuming it to be compatible with latest v2.


HRNet_SEG(
  (backbone): HighResolutionNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True,

In [3]:
# model = model.cuda()

t = torch.randn([1, 3, 600, 300]).cuda()

o = model(t)

o.shape

torch.Size([1, 15, 150, 75])

In [1]:
from torchmetrics.functional import dice
